In [ ]:
import os,re
import json
import time
import math 
import httplib2
import requests
import pinecone 
import pandas as pd
import numpy as np
import urllib.request
from bs4 import BeautifulSoup
from bs4.element import Comment
from youtubesearchpython import *
from langchain.llms import OpenAIChat
from bs4 import BeautifulSoup, SoupStrainer
from langchain.vectorstores import Pinecone
from langchain.chains import VectorDBQAWithSourcesChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Shenyen GPT

- Scrape source from: https://miblue119.github.io/shengyen_lectures/index.html
- Chunk data 
- Embed it to Pineconde
- Test VectorDBQA chain on it

`1. Scrape transcriptions -` 

In [ ]:
# Use the code based on Lex-GPT's script
# Get text -
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(string=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def get_text_and_title(url):
    html = urllib.request.urlopen(url).read()
    t=(text_from_html(html))
    title=t.split("|")[0].split("back to index")[1].strip("link ")
    return t, title

# Get links -
def get_links(URL):
    http = httplib2.Http()
    status, response = http.request(URL)
    links = []
    for link in BeautifulSoup(response, 'html.parser', parse_only=SoupStrainer('a')):
        if link.has_attr('href'):
            links.append(link['href'])
    links_clean = [l for l in links if "https" in l]
    return links_clean

# Get image -
def get_img(URL,title,episode_id):
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    img_tags = soup.find_all('img')
    urls = [img['src'] for img in img_tags]
    for url in urls:
        response = requests.get(url)
        imgpath="../public/0%s.jpg"%episode_id
        with open(imgpath, 'wb') as f:
            if 'http' not in url:
                url = '{}{}'.format(site, url)
            response = requests.get(url)
            f.write(response.content)
    return imgpath

# Full pipeline - 
def pre_process(URL,episode_id):

    t,title=get_text_and_title(URL)
    # print(f"Title: {title}")
    # print(f"Episode ID: {episode_id}")
    links=get_links(URL)
    # print(f"Links: {len(links)}")
    img=get_img(URL,title,episode_id)
    stor_chunk = pd.DataFrame()
    stor_chunk['chunks']= t.split("link |")
    # print(f"stor_chunk['chunks']: {stor_chunk['chunks']}")
    #stor_chunk['clean_chunks']=stor_chunk['chunks'].apply(lambda x: re.sub(r"[^a-zA-Z ]+", '', x)).apply(lambda x: x.strip())
    stor_chunk['clean_chunks'] = stor_chunk['chunks'].apply(lambda x: re.sub(r"[^a-zA-Z\u4e00-\u9fff ]+", '', x)).apply(lambda x: x.strip())
    # print(f"stor_chunk['clean_chunks']: {stor_chunk['clean_chunks']}")
    stor_chunk['links']=links
    # print(f"stor_chunk['links']: {stor_chunk['links']}")
    all_text = stor_chunk['clean_chunks'].str.cat(sep=' ')
    # print(f"all_text: {all_text}")
    return all_text, links, title

# Make splits - 
def make_splits(chunks, overlap_scale, URL):

    # ID
    # 'https://miblue119.github.io/shengyen_lectures/0001/0000/0000.html'
    # Get the last 3 parts of the URL and generate id to 0001_0000_0000
    episode_id='_'.join(URL.strip('.html').split('/')[-3:])

    # Pre-processing
    texts,links,title=pre_process(URL,episode_id)
    print(texts)
    
    # Splits 
    chunk_overlap = int(chunks * overlap_scale)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunks, 
                                                   chunk_overlap=chunk_overlap) 
    texts_recusive = text_splitter.split_text(texts)
    print(len(texts_recusive)) 

    # Metadata 
    N = len(texts_recusive) 
    bins = np.linspace(0, len(links)-1, N, dtype=int)
    sampled_links = [links[i] for i in bins]
    # Here we can add "link", "title", etc that can be fetched in the app 
    metadatas=[{"source":title + " " +link,"id":episode_id,"link":link,"title":title} for link in sampled_links]
    print(len(metadatas))
    return texts_recusive,metadatas,title,episode_id

In [ ]:
# Get all pages 
http = httplib2.Http()
status, response = http.request("https://miblue119.github.io/shengyen_lectures/index.html")
links = []
for link in BeautifulSoup(response, 'html.parser', parse_only=SoupStrainer('a')):
    if link.has_attr('href'):
        links.append(link['href'])
links_tx = ["https://miblue119.github.io/shengyen_lectures/"+l for l in links if "0" in l]

In [ ]:
def get_subtitiles_page_links(index_page_url):
    # Retrieve the lecture index page
    http = httplib2.Http()
    status, response = http.request(index_page_url)
    links = []
    for link in BeautifulSoup(response, 'html.parser', parse_only=SoupStrainer('a')):
        if link.has_attr('href'):
            links.append(link['href'])
    lectures_base_url = index_page_url.split("index.html")[0] 
    links_lectures = [lectures_base_url+l for l in links if "0" in l]
    # Remove the link ending without `.html`
    links_lectures = [l for l in links_lectures if l.endswith('.html')]
    # Retrieve the lecture subtitiles page
    links_subtitles = []
    for link_lecture in links_lectures:
        #print(link_lecture)
        status, response = http.request(link_lecture)
        subtile_links_candidates = []
        for link in BeautifulSoup(response, 'html.parser', parse_only=SoupStrainer('a')):
            if link.has_attr('href'):
                subtile_links_candidates.append(link['href'])
        subtile_links = [l for l in subtile_links_candidates if "0" in l]
        # Remove the link ending without `.html`
        subtile_links = [l for l in subtile_links if l.endswith('.html')]
        
        prefix_link_lecture = "/".join(link_lecture.split("/")[:-1])
        subtile_links = [prefix_link_lecture + "/" + l for l in subtile_links]
        links_subtitles.extend(subtile_links)
    return links_subtitles

In [ ]:
links_subtitles = get_subtitiles_page_links("https://miblue119.github.io/shengyen_lectures/index.html")

In [ ]:
# *** Chunk size: key parameter *** 
import os 
chunks = 200
overlap_scale = 0.2
splits_scrape = [ ]
metadatas_scrape = [ ]
 
transcripts_dir = "docs"
metadatas_dir = "metadatas"
os.makedirs(transcripts_dir, exist_ok=True)
os.makedirs(metadatas_dir, exist_ok=True)

# Iterate 
stor=pd.DataFrame()
for page in links_subtitles:
    try:
        print("Writing: %s"%page)
        # Make splits
        splits,metadatas,title,episode_id=make_splits(chunks,overlap_scale, page)
        stor.loc[episode_id,'title']=title 
        with open(f'{transcripts_dir}/%s.txt'%episode_id, "w") as f:
            for string in splits:
                f.write(string + "\n") 
        f.close()
        with open(f'{metadatas_dir}/%s.json'%episode_id, "w") as f:
            # Dump the json with indent=4
            json.dump(metadatas, f, indent=4)
        f.close()
        splits_scrape.append(splits)
        metadatas_scrape.append(metadatas)
    except Exception as e:
        print(e)
        print("Error on page: %s"%page)

In [ ]:
len(splits_scrape), len(metadatas_scrape)

`2. Embed full dataset in Pinecone VectorDB -`

In [ ]:
# Join the list of lists 
splits_all = []
# For the initial write 
# for sublist in [splits_scrape+splits_new]:
# For updates -- 
for sublist in splits_scrape:
    splits_all.extend(sublist)

metadatas_all = []
# For the initial write 
# for sublist in [metadatas_scrape+metadatas_new]:
# For updates -- 
for sublist in metadatas_scrape:
    metadatas_all.extend(sublist)

In [ ]:
len(splits_all), len(metadatas_all)

In [ ]:
# Pinecone
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY'),  
    environment="us-east1-gcp"  
)
index_name = "shenyen-gpt"
embeddings = OpenAIEmbeddings()

# Initialize with small set of data - 
p = Pinecone.from_texts(splits_all[0:2], 
                           embeddings, 
                           index_name=index_name, 
                           metadatas=metadatas_all[0:2])

# Update - 

# embeddings = OpenAIEmbeddings()
# p = Pinecone.from_existing_index(index_name=index_name,embedding=embeddings)

In [ ]:
# Add data in chunk to avoid data ingest errors
chunk_size = 100
last_chunk = 0
num_chunks = math.ceil(len(splits_all) / chunk_size)
for i in range(last_chunk,num_chunks):
    
    print(i)
    start_time = time.time()
    start_idx = i * chunk_size
    end_idx = min(start_idx + chunk_size, len(splits_all))
    
    # Extract the current chunk
    current_splits = splits_all[start_idx:end_idx]
    current_metadatas = metadatas_all[start_idx:end_idx]
    
    # Add the current chunk to the vector database
    p.add_texts(texts = current_splits, metadatas=current_metadatas)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    print("--------")

`3. Read in VectorDB for testing` 

In [ ]:
# Pinecone
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY'),  
    environment="us-east1-gcp"  
)
index_name = "shenyen-gpt"
embeddings = OpenAIEmbeddings()
p = Pinecone.from_existing_index(index_name=index_name,embedding=embeddings)

`4. Run VectorDBQAWithSourcesChain`

In [ ]:
def run_vectordb_sources_chain(llm,query,docstore):
    
    start_time = time.time()
    chain = VectorDBQAWithSourcesChain.from_chain_type(llm, chain_type="stuff", vectorstore=docstore)
    a = chain({"question": query},return_only_outputs=True)
    print(a["answer"])
    print(a["sources"])
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    print("--------")

llm = OpenAIChat(temperature=0)
q = "學佛可以幹嘛?#lang:zh-Hant"
run_vectordb_sources_chain(llm,q,p)